# AutoGluon Multimodal Model

Using AutoGluon's TabularPredictor with multimodal features:
- Text features: request_title, request_text, request_text_edit_aware
- Categorical: requester_user_flair
- Numeric: all other features
- Handles class imbalance automatically
- Uses ensemble of multiple models including text transformers

In [3]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Create experiments directory
Path('/home/code/experiments').mkdir(parents=True, exist_ok=True)

print("Loading data...")

Loading data...


In [4]:
# Load training data - file appears to be a JSON array, not line-delimited
import json

with open('/home/data/train.json', 'r') as f:
    # Try to load as a single JSON array
    try:
        train_data = json.load(f)
    except:
        # If that fails, try line-delimited
        f.seek(0)
        train_data = [json.loads(line) for line in f if line.strip()]

train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")

# Load test data
with open('/home/data/test.json', 'r') as f:
    try:
        test_data = json.load(f)
    except:
        f.seek(0)
        test_data = [json.loads(line) for line in f if line.strip()]

test_df = pd.DataFrame(test_data)
print(f"Test data shape: {test_df.shape}")

# Check target distribution
target = 'requester_received_pizza'
print(f"\nTarget distribution:")
print(train_df[target].value_counts())
print(f"Positive rate: {train_df[target].mean():.3f}")

Training data shape: (4040, 32)
Test data shape: (1631, 17)

Target distribution:
requester_received_pizza
False    3046
True      994
Name: count, dtype: int64
Positive rate: 0.246


In [5]:
# Install AutoGluon if not available
try:
    from autogluon.tabular import TabularPredictor
    print("AutoGluon is available")
except ImportError:
    print("Installing AutoGluon...")
    !pip install -q autogluon
    from autogluon.tabular import TabularPredictor
    print("AutoGluon installed successfully")

AutoGluon is available


In [11]:
from autogluon.tabular import TabularPredictor

# Prepare data - AutoGluon can handle text directly
# Need to convert list columns to strings first

# Convert list columns to string representation
list_col = 'requester_subreddits_at_request'
if list_col in train_df.columns:
    train_df[list_col] = train_df[list_col].apply(lambda x: ','.join(x) if isinstance(x, list) else str(x))
    test_df[list_col] = test_df[list_col].apply(lambda x: ','.join(x) if isinstance(x, list) else str(x))

# Define feature columns (exclude ID columns and target)
exclude_cols = ['request_id', 'requester_received_pizza', 'giver_username_if_known']
feature_cols = [col for col in train_df.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")
print(f"Sample features:")
for col in feature_cols[:5]:
    print(f"  {col}: {train_df[col].dtype}")

Using 29 features
Sample features:
  number_of_downvotes_of_request_at_retrieval: int64
  number_of_upvotes_of_request_at_retrieval: int64
  post_was_edited: object
  request_number_of_comments_at_retrieval: int64
  request_text: object


In [12]:
# Investigate data types and potential issues
print("Data types:")
print(train_df[feature_cols].dtypes.value_counts())

# Check for object columns that might contain lists
object_cols = train_df[feature_cols].select_dtypes(include=['object']).columns
print(f"\nObject columns ({len(object_cols)}):")
for col in object_cols:
    sample_val = train_df[col].iloc[0]
    print(f"  {col}: {type(sample_val)} - {str(sample_val)[:100]}...")
    
# Check for list columns
list_cols = []
for col in feature_cols:
    if isinstance(train_df[col].iloc[0], list):
        list_cols.append(col)
        
if list_cols:
    print(f"\nColumns containing lists: {list_cols}")
else:
    print("\nNo list columns found")

Data types:
int64      16
object      7
float64     6
Name: count, dtype: int64

Object columns (7):
  post_was_edited: <class 'bool'> - False...
  request_text: <class 'str'> - Hi I am in need of food for my 4 children we are a military family that has really hit hard times an...
  request_text_edit_aware: <class 'str'> - Hi I am in need of food for my 4 children we are a military family that has really hit hard times an...
  request_title: <class 'str'> - Request Colorado Springs Help Us Please...
  requester_subreddits_at_request: <class 'str'> - ...
  requester_user_flair: <class 'NoneType'> - None...
  requester_username: <class 'str'> - nickylvst...

No list columns found


In [13]:
# Configure AutoGluon for class imbalance
# Use 'best_quality' preset for maximum performance
# Set time limit to avoid running too long

predictor = TabularPredictor(
    label=target,
    problem_type='binary',
    eval_metric='roc_auc',  # Good for imbalanced classification
    path='/home/code/experiments/autogluon_models'
).fit(
    train_data=train_df[feature_cols + [target]],
    presets='best_quality',
    time_limit=1200,  # 20 minutes
    verbosity=2
)

Verbosity: 2 (Standard Logging)


=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #21~24.04.1-Ubuntu SMP Fri Oct 17 00:56:30 UTC 2025
CPU Count:          12
Pytorch Version:    2.9.1+cu128
CUDA Version:       12.8
GPU Memory:         GPU 0: 79.25/79.25 GB
Total GPU Memory:   Free: 79.25 GB, Allocated: 0.00 GB, Total: 79.25 GB
GPU Count:          1
Memory Avail:       161.45 GB / 167.04 GB (96.7%)
Disk Space Avail:   940.04 GB / 3389.36 GB (27.7%)


Presets specified: ['best_quality']


Using hyperparameters preset: hyperparameters='zeroshot'


Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)


...

	1.0	 = Validation score   (roc_auc)


	0.27s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 891.77s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 198.3 rows/s (505 batch size)


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/code/experiments/autogluon_models")


In [ ]:
# Generate predictions on test set
test_predictions = predictor.predict_proba(test_df[feature_cols])

# Get the probability for the positive class (True)
if isinstance(test_predictions, pd.DataFrame):
    # AutoGluon returns a DataFrame with columns for each class
    positive_class = predictor.positive_class
    test_pred_proba = test_predictions[positive_class]
else:
    # If it's a Series or array
    test_pred_proba = test_predictions

print(f"Test predictions shape: {test_pred_proba.shape}")
print(f"Test predictions sample:\n{test_pred_proba.head()}")

# Get leaderboard to see model performance
leaderboard = predictor.leaderboard(silent=True)
print("\nLeaderboard (top 5 models):")
print(leaderboard.head())

# Get the best model's validation score
best_model_score = leaderboard.iloc[0]['score_val']
print(f"\nBest model validation score: {best_model_score:.4f}")

# Create submission
submission_df = pd.DataFrame({
    'request_id': test_df['request_id'],
    'requester_received_pizza': test_pred_proba
})

print(f"\nSubmission shape: {submission_df.shape}")
print(f"Submission sample:\n{submission_df.head()}")

# Save submission
submission_path = '/home/submission/autogluon_submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"\nSubmission saved to: {submission_path}")

In [ ]:
# Make predictions on test set
test_predictions = predictor.predict_proba(test_df[feature_cols])[[1]]  # Get probability of positive class

# Create submission
submission = pd.DataFrame({
    'request_id': test_df['request_id'],
    'requester_received_pizza': test_predictions
})

print("Submission preview:")
print(submission.head())
print(f"\nSubmission shape: {submission.shape}")

# Check prediction distribution
print(f"\nPrediction distribution:")
print(f"Mean: {submission['requester_received_pizza'].mean():.4f}")
print(f"Std: {submission['requester_received_pizza'].std():.4f}")
print(f"Min: {submission['requester_received_pizza'].min():.4f}")
print(f"Max: {submission['requester_received_pizza'].max():.4f}")

# Save submission
submission_path = '/home/submission/submission_autogluon.csv'
submission.to_csv(submission_path, index=False)
print(f"\nSubmission saved to: {submission_path}")